# Speckle aberration recovery

Gautam Gunjala & Antoine Wojdyla, June 2015

In [ ]:
import numpy as np

## Load data

In [ ]:
# display data

## Pre-process data

In [ ]:
# whitening 

## Extract aberration

In [ ]:
# display results